In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Mediapipe Setup
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Global State Variables
counter = 0
stage = None
start_clicked = False
workout_clicked = False
countdown_started = False
start_time = None
countdown = 5
body_part = None
workout = None

# Button Coordinates
restart_button_coords = (20, 80, 140, 130)
chest_button_coords = (20, 140, 140, 190)
leg_button_coords = (20, 200, 140, 250)
back_button_coords = (20, 260, 140, 310)

# Workout Buttons
buttons = {
    "Chest": {
        "pushup": (150, 140, 270, 190),
        "bicep_curl": (150, 200, 270, 250)
    },
    "Leg": {
        "squat": (150, 140, 270, 190),
        "calf_raises": (150, 200, 270, 250),
        "rdl": (150, 260, 270, 310)
    },
    "Back": {
        "shoulder_presses": (150, 140, 270, 190),
        "shoulder_raises": (150, 200, 270, 250)
    }
}

# Workout Joint Mapping
def get_joint_info(workout, landmarks):
    mp_lm = mp_pose.PoseLandmark
    wmap = {
        "pushup":       ([mp_lm.LEFT_SHOULDER, mp_lm.LEFT_ELBOW, mp_lm.LEFT_WRIST], 140, 90),
        "bicep_curl":   ([mp_lm.LEFT_SHOULDER, mp_lm.LEFT_ELBOW, mp_lm.LEFT_WRIST], 160, 45),
        "squat":        ([mp_lm.LEFT_HIP, mp_lm.LEFT_KNEE, mp_lm.LEFT_ANKLE], 160, 135),
        "calf_raises":  ([mp_lm.LEFT_HEEL, mp_lm.LEFT_FOOT_INDEX, None], 170, 160),
        "rdl":          ([mp_lm.LEFT_SHOULDER, mp_lm.LEFT_HIP, mp_lm.LEFT_ANKLE], 160, 110),
        "shoulder_presses": ([mp_lm.LEFT_SHOULDER, mp_lm.LEFT_ELBOW, mp_lm.LEFT_WRIST], 150, 100),
        "shoulder_raises":  ([mp_lm.LEFT_HIP, mp_lm.LEFT_SHOULDER, mp_lm.LEFT_ELBOW], 70, 20)
    }

    if workout not in wmap:
        return None, None, None, None

    joints, rest_angle, rep_angle = wmap[workout]
    a = [landmarks[joints[0].value].x, landmarks[joints[0].value].y]
    b = [landmarks[joints[1].value].x, landmarks[joints[1].value].y]
    if joints[2] is None:
        c = [0, landmarks[joints[1].value].y]
    else:
        c = [landmarks[joints[2].value].x, landmarks[joints[2].value].y]

    return a, b, c, (rest_angle, rep_angle)

# Angle Calculation
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180.0 else angle

# Mouse Interaction
def mouse_callback(event, x, y, flags, param):
    global counter, stage, start_clicked, workout_clicked, countdown_started, start_time, body_part, workout

    if event != cv2.EVENT_LBUTTONDOWN:
        return

    # Restart
    if inside(x, y, restart_button_coords):
        counter = 0
        stage = None
        start_clicked = False
        workout_clicked = False
        return

    # Body Part Selection
    for part, coords in zip(["Chest", "Leg", "Back"],
                            [chest_button_coords, leg_button_coords, back_button_coords]):
        if inside(x, y, coords):
            start_clicked = True
            body_part = part
            return

    # Workout Selection
    if start_clicked and body_part:
        for w_name, coords in buttons[body_part].items():
            if inside(x, y, coords):
                workout = w_name
                countdown_started = True
                workout_clicked = True
                start_time = time.time()
                return

def inside(x, y, coords):
    x1, y1, x2, y2 = coords
    return x1 <= x <= x2 and y1 <= y <= y2

# Main App
cap = cv2.VideoCapture(0)
cv2.namedWindow("Mediapipe Feed")
cv2.setMouseCallback("Mediapipe Feed", mouse_callback)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # UI State
        if not start_clicked:
            for label, coords in zip(["Chest", "Leg", "Back"], [chest_button_coords, leg_button_coords, back_button_coords]):
                x1, y1, x2, y2 = coords
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 204, 102), -1)
                cv2.putText(image, label, (x1 + 20, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.imshow("Mediapipe Feed", image)
            if cv2.waitKey(10) & 0xFF == ord('q'): break
            continue

        if body_part and not workout_clicked:
            for name, coords in buttons[body_part].items():
                x1, y1, x2, y2 = coords
                label = name.replace("_", " ").title()
                cv2.rectangle(image, (x1, y1), (x2, y2), (220, 172, 39), -1)
                cv2.putText(image, label, (x1 + 10, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.imshow("Mediapipe Feed", image)
            if cv2.waitKey(10) & 0xFF == ord('q'): break
            continue

        if countdown_started:
            remaining = countdown - int(time.time() - start_time)
            if remaining > 0:
                cv2.putText(image, f"Starting in {remaining}",
                            (image.shape[1]//2 - 150, image.shape[0]//2),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
                cv2.imshow("Mediapipe Feed", image)
                if cv2.waitKey(10) & 0xFF == ord('q'): break
                continue

        # Pose Detection & Angle Logic
        if results.pose_landmarks and workout:
            landmarks = results.pose_landmarks.landmark
            joint_a, joint_b, joint_c, angles = get_joint_info(workout, landmarks)

            if joint_a and joint_b and joint_c and angles:
                angle = calculate_angle(joint_a, joint_b, joint_c)
                rest_angle, rep_angle = angles
                elbow_coords = tuple(np.multiply(joint_b, [image.shape[1], image.shape[0]]).astype(int))
                cv2.putText(image, str(round(angle, 2)), elbow_coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

                if angle > rest_angle:
                    stage = "down"
                if angle < rep_angle and stage == "down":
                    stage = "up"
                    counter += 1
                    print(counter)

        # UI Overlays
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2)

        # Restart Button
        x1, y1, x2, y2 = restart_button_coords
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 122, 204), -1)
        cv2.putText(image, "Restart", (x1 + 10, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

        # Landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(51, 39, 225), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))

        cv2.imshow("Mediapipe Feed", image)
        if cv2.waitKey(10) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1764026371.282007 54813595 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
W0000 00:00:1764026371.368228 54818335 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764026371.390079 54818341 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1
1
2
3
4
